In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = 30
pd.options.display.max_rows = 1050


creating the dataframe

In [ ]:
def openFile(x, detentionCenter):
    file = pd.read_csv(
        r"/Users/stjames/Dropbox/Pablo/data/weekly data transfer/" + str(detentionCenter) + "/" + str(x) + ".csv",
        header=0, index_col='record_id',usecols=['record_id', str(x)])
    file = file.loc[~file.index.duplicated(),:]
    
    return file

def openMVDFFile(x):
    return pd.read_csv(
        r"/Users/stjames/Dropbox/Pablo/data/weekly data transfer/mvdf/" + str(x) + ".csv",
        header=0, usecols=['record_id', 'comment'])

def formMVDF_t():
    # this will be the column to demonstrate transfers
    names = ['gsa', 'golden', 'transferGsa', 'transferGolden']

    gsa, golden, transferGsa, transferGolden = [openMVDFFile(x) for x in names]

    gsaJoin = pd.concat([gsa, transferGsa])
    goldenJoin = pd.concat([golden, transferGolden])
    mvdf_t = pd.concat([goldenJoin,gsaJoin])
    mvdf_t = mvdf_t.drop_duplicates()
    mvdf_t.reset_index(inplace=True)
    mvdf_t.drop(labels='index', axis=1, inplace=True)

    return mvdf_t

def case_event(detentionCenter):
    
    casetype_open = pd.read_csv(r"/Users/stjames/Dropbox/Pablo/data/weekly data transfer/" + str(detentionCenter) + "/" +"casetype_open.csv",
                                usecols=['record_id', 'casetype_id', 'opened', 'closed', 'caseregisters_id'])
    casetype_closed = pd.read_csv(r"/Users/stjames/Dropbox/Pablo/data/weekly data transfer/" + str(detentionCenter) + "/" +"casetype_closed.csv",
                                usecols=['record_id', 'casetype_id', 'opened', 'closed', 'caseregisters_id'])
    df = pd.concat([casetype_open, casetype_closed])
    df = df.set_index(['record_id','casetype_id'])
    df.sort_index(inplace=True)
    df = df.loc[~df.index.duplicated(), :]
    
    return df
    

def openOG(detentionCenter):
    df = pd.read_csv(
        r"/Users/stjames/Dropbox/Pablo/data/weekly data transfer/" + str(detentionCenter) + "/opened.csv",
        header=0, usecols=['record_id', 'firstName', 'lastName', 'caseaction_id'])
    df = df.loc[~df.record_id.duplicated(), :]
    return df

def combine(detentionCenter, colsStr):
    opened = openOG(detentionCenter)
    opened = opened.set_index('record_id')
    case = case_event(detentionCenter)
    aNum, address, city, county, state, zip, gender_id, DOB, birth_country_id, ethnicity, race, language_id, immigration_status_id, initiated_date, pob = [
        openFile(x, detentionCenter) for x in colsStr]
    # this was a great way to open many files at once
    cols = aNum, address, city, county, state, zip, gender_id, DOB, birth_country_id, ethnicity, race, language_id, immigration_status_id, initiated_date, pob

    joined = opened.join(cols)
    joined = joined.reset_index()
    #joined['record_id'] = joined['index']
    #joined.drop(labels='index', axis=1, inplace=True)
    joined = joined.loc[~joined.record_id.duplicated(), :]
    
    case = case.reset_index()
    
    df = case.merge(joined, on='record_id')
    
    return df

def mvdfCombine(mvdf_t, detentionCenter, colsStr):
    transfers = mvdf_t.set_index('record_id')
    opened = openOG(detentionCenter)
    opened = opened.set_index('record_id')
    case = case_event(detentionCenter)
    aNum, address, city, county, state, zip, gender_id, DOB, birth_country_id, ethnicity, race, language_id, immigration_status_id, initiated_date, pob = [
        openFile(x, detentionCenter) for x in colsStr]
    # this was a great way to open many files at once
        
    cols = aNum, address, city, county, state, zip, gender_id, DOB, birth_country_id, ethnicity, race, language_id, immigration_status_id, initiated_date, pob, transfers

    joined = opened.join(cols)
    joined = joined.reset_index()
    #joined['record_id'] = joined['index']
    #joined.drop(labels='index', axis=1, inplace=True)
    joined = joined.loc[~joined.record_id.duplicated(), :]
        
    case = case.reset_index()
    df = case.merge(joined, on='record_id')
    
    return df

In [93]:
colsStr = ["aNum", "address", "city", "county", "state", "zip", "gender_id", "DOB", "birth_country_id", "ethnicity",
           "race", "language_id", "immigration_status_id", "initiated_date", "pob"]
corCol = {'opened':'lawlab_opened', 'firstName':'first_name', 'lastName':'last_name', 'caseaction_id':'cohort', 'aNum':'a_number', 'gender_id':'gender', 'birth_country_id':'country_of_birth', 'language_id':'language', 'immigration_status_id':'immigration_status', 'initiated_date':'date_detained', 'closed':'lawlab_closed', 'comment':'transfers', 'casetype_id':'casetype'}
corCity = {'Pamona': 'Pomona', 'Azuza': 'Azusa', 'Monte': 'El Monte', 'El Puente': 'La Puente', 'Santa Barbera': 'Santa Barbera', 'San Francico': 'San Francisco', 'Anahaim': 'Anaheim', 'Colinga': 'Coalinga', 'Westminister': 'Westminster', 'Fondana': 'Fontana', 'Vajello': 'Vcltejo', 'Pittsburgh': 'Pittsburg',}
corCountry = {'CHINA':'China, Peoples Republic of', "LAO PEOPLE'S DEMOCRATIC REPUBLIC":'Laos', 'MOLDOVA, REPUBLIC OF':'Moldova', 'AMERICAN SAMOA':'Samoa', 'CZECH REPUBLIC':'Czechia (Czech Republic)'}
corCohort = {'@Golden State':'GOLDEN STATE ANNEX', '@MVDF':'Mesa Verde Detention Facility', '@Yuba':'YUBA COUNTY FACILITY'}
corImm = {'No Lawful Status / No Estatus Legal':'Undocumented', 'LPR (Lawful Permanent Resident) / (Residente Legal Permanente)':'LPR', 'Unknown / Desconocidx':'Unknown', 'Other / Otro':'Other', 'Asylee / Asiladx':'Asylee'}
corGender = {'Non-binary':'Nonbinary'}
corCasetype = {'Golden State KYR - Golden State Detention Visits':'GSA_KYR', 'MVDF KYR - Detention Visit (Centro)':'MVDF_KYR_CENTRO', 
               'AOD - Detained Attorney of the Day':'AOD', 'CCIJ Yuba KYR - Yuba Detention Visits':'YCJ_KYR', 'Hotline - CCIJ Hotline Calls':'HOTLINE',
               'RO-VD (AOD) - AOD Removals/Voluntary Departure':'RO_VD_AOD', 'Referrals - Referrals to CCIJ':'REFERRALS', 'RAPID RESPONSE - Regional Rapid Response':'RRR',
               'AB32 MVDF - AB32 Coordination':'AB32_MVDF', 'SFPD - SF Public Defender':'SFPD'}
corCounty = {'San Bernadino':'San Bernardino'}

In [94]:
mvdf_t = formMVDF_t()
mvdf = mvdfCombine(mvdf_t, "mvdf", colsStr)
yuba = combine("yuba", colsStr)


In [95]:
gsa = combine("gsa", colsStr)


In [96]:
clt = pd.concat([gsa, mvdf, yuba])
clt.reset_index(inplace=True)
clt.drop(labels='index', axis=1, inplace=True)

In [97]:
clt = clt.rename(mapper=corCol, axis=1)
clt['a_number'] = clt['a_number'].astype('str')

clt['county']=clt['county'].str.title()
clt['county'] = clt['county'].where(clt['county']!="Unknown")
clt['city']=clt['city'].str.title()
clt['pob']=clt['pob'].str.title()
clt['zip']=clt['zip'].astype('str')

clt.loc[~clt.county.isna(),'county'] = clt.loc[~clt.county.isna(),'county'].replace(to_replace='(\s+County)', value='', regex=True).replace(to_replace='(\s+County)', value='', regex=True)

clt['city'].replace(to_replace=corCity, value=None, inplace=True)
clt['county'].replace(to_replace=corCounty, value=None, inplace=True)
clt['country_of_birth'].replace(to_replace=corCountry, value=None, inplace=True)
clt['cohort'].replace(to_replace=corCohort, value=None, inplace=True)
clt['immigration_status'].replace(to_replace=corImm, value=None, inplace=True)  
clt['gender'].replace(to_replace=corGender, value=None, inplace=True)
clt['casetype'].replace(to_replace=corCasetype, value=None, inplace=True)
clt['presumed_detained'] = clt['lawlab_closed']
clt['presumed_detained'].replace(to_replace=r'(.+)', value=False, inplace=True, regex=True)
clt['presumed_detained'].fillna(value=True, inplace=True)

clt['lawlab_opened'] = pd.to_datetime(clt['lawlab_opened'])
clt['DOB'] = pd.to_datetime(clt['DOB'])



In [98]:
clt = clt.set_index(['record_id','casetype'])

In [99]:
casetypes = ['GSA_KYR','MVDF_KYR_CENTRO','AOD', 'YCJ_KYR', 'HOTLINE', 'RO_VD_AOD', 'REFERRALS', 'RRR', 'AB32_MVDF', 'SFPD']
index_slice = ['GSA_KYR','AOD', 'YCJ_KYR', 'HOTLINE', 'RO_VD_AOD', 'REFERRALS', 'RRR', 'AB32_MVDF', 'SFPD']

In [100]:
no_centro = clt.loc[(slice(None), index_slice), :]

In [101]:
kk_gsa = (clt['cohort']=='GOLDEN STATE ANNEX') & (clt['lawlab_opened']>='2020-09-01')
kk_mvdf = (clt['cohort']=='Mesa Verde Detention Facility') & (~clt['transfers'].isna())
kk_ycj = (clt['cohort']=='YUBA COUNTY FACILITY') & (clt['lawlab_opened']>='2020-01-01')

In [102]:
kk_crit = clt[kk_gsa | kk_mvdf | kk_ycj]

In [103]:
df = kk_crit.reset_index()

In [104]:
df = df.loc[:,['record_id','caseregisters_id','first_name','last_name','a_number']]

creating the folders

In [105]:
import os

In [106]:
def create_folder(df):
    parent_dir = '/Users/stjames/Dropbox/Pablo/data/ll_batch_download'
    for x in df.record_id.values:
        path = os.path.join(parent_dir,str(x))
        os.mkdir(path)
        

change the markdown below to code when ready to do again

create_folder(df)

downloading the file

In [107]:
df = df.set_index('record_id')

In [108]:
df = df[:20]

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os

In [ ]:
os.getcwd()

In [ ]:
chromedriver = '/Users/stjames/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver)

In [ ]:
for x,y in df.caseregisters_id.items():
        folder = '/Users/stjames/Dropbox/Pablo/data/ll_batch_download/'+str(x)
        url = 'https://ccij-cdb.innovationlawlab.org/records/'+str(x)+'/case/'+str(y)
        op = Options()
        op.add_experimental_option('prefs',{'download.default_directory':folder})
        chromedriver = '/Users/stjames/Documents/chromedriver'
        driver = webdriver.Chrome(chromedriver, chrome_options=op)
        driver.implicitly_wait(10)
        driver.get(url)
        driver.find_element(By.XPATH,'//*[@id="user_login"]').send_keys('pablo-ccij')
        driver.find_element(By.XPATH,'//*[@id="user_password"]').send_keys('Clement3!')
        driver.find_element(By.XPATH,'//*[@id="loginform"]/button/span').click()
        time.sleep(3)
        driver.find_element(By.XPATH, '//div/div/div[3]/button').click()
        time.sleep(.5)
        driver.find_element(By.XPATH, '//div/main/div/div/div[2]/div/div[2]/nav/div/div[3]').click()
        time.sleep(.5)
        driver.find_element(By.XPATH,'//div/div[2]/a').click()
        time.sleep(15)

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from csv import DictReader
import pickle
import pprint

In [ ]:
cookies_location = 'cookies.txt'

def save_cookies(driver, location):

    pickle.dump(driver.get_cookies(), open(location, "wb"))


def load_cookies(driver, location, url):

    cookies = pickle.load(open(location, "rb"))
    driver.delete_all_cookies()
    # have to be on a page before you can add any cookies, any page - does not matter which
    driver.get(url)
    for cookie in cookies:
        if isinstance(cookie.get('expiry'), float):#Checks if the instance expiry a float 
            cookie['expiry'] = int(cookie['expiry'])# it converts expiry cookie to a int 
        driver.add_cookie(cookie)


def delete_cookies(driver, domains=None):

    if domains is not None:
        cookies = driver.get_cookies()
        original_len = len(cookies)
        for cookie in cookies:
            if str(cookie["domain"]) in domains:
                cookies.remove(cookie)
        if len(cookies) < original_len:  # if cookies changed, we will update them
            # deleting everything and adding the modified cookie object
            driver.delete_all_cookies()
            for cookie in cookies:
                driver.add_cookie(cookie)
    else:
        driver.delete_all_cookies()

In [ ]:
# Initial load of the domain that we want to save cookies for

url = 'https://ccij-cdb.innovationlawlab.org/records/20220193/case/5686'

chromedriver = '/Users/stjames/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get(url)
driver.find_element(By.XPATH,'//*[@id="user_login"]').send_keys('pablo-ccij')
driver.find_element(By.XPATH,'//*[@id="user_password"]').send_keys('Clement3!')
driver.find_ele5ment(By.XPATH,'//*[@id="loginform"]/button/span').click()

save_cookies(driver, cookies_location)
driver.quit()

In [ ]:
driver = webdriver.Chrome(chromedriver)
load_cookies(driver, cookies_location, url)
driver.get(url)
time.sleep(3)
driver.get(url)

In [ ]:
import zipfile
import os

In [ ]:
os.chdir('/Users/stjames/Dropbox/Pablo/data/batch_download_test')

In [ ]:
from zipfile import ZipFile

In [ ]:
for x in os.listdir():
    if x == '.DS_Store':
        pass
    else:
        with ZipFile('/Users/stjames/Dropbox/Pablo/data/batch_download_test/'+x+'/download_all.zip', 'r') as zip_object:
            zip_object.extractall(path='/Users/stjames/Dropbox/Pablo/data/batch_download_test/'+x+'/')

In [ ]:
for x in os.listdir():
    if x == '.DS_Store':
        pass
    else:
        os.remove('/Users/stjames/Dropbox/Pablo/data/batch_download_test/'+x+'/download_all.zip')
                  

In [ ]:
cols = ['Contact.ExternalID', 'Body']
df2 = pd.DataFrame(columns=cols)

In [ ]:
for x, y in df.first_name.items():
    df['Contact.ExternalID'][x]=str(x)
for x in os.listdir():
    df['Body'][x]='batch_download_test/'+str(x)+'/24982'

In [121]:
df2 = df.loc[:,['first_name', 'last_name']]
df2.index = df2.index.astype(str)

In [ ]:
import re

In [133]:
for top, files, dirs in os.walk(os.getcwd()):
    if len(top) == 68:
        m = re.search('\d{8}', top)
        r = m.group(0)
        df2.loc[r, 'Contact.ExternalID'] = r
        df2.loc[r,'Body'] = top
        

In [135]:
df2.to_csv('/Users/stjames/Dropbox/Pablo/data/batch_download.csv')

In [ ]:
len('/Users/stjames/Dropbox/Pablo/data/batch_download_test/20200550/25058')